## Вторая задача
Для второй задачи был подготовлен отдельно датасет
В данной задаче не было разбивки на кагорты, поэтому мы их не учитываем при подготовке датасета
Используем фэковые данные, с добавлением кейсов, когда есть маленькие интервалы между уроками 


In [5]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

# делаем фэйковые данные 
fake = Faker()

professions = [
    {'profession_id': 1, 'profession_name': 'Data Analyst'},
    {'profession_id': 2, 'profession_name': 'Software Engineer'},
    {'profession_id': 3, 'profession_name': 'Graphic Designer'},
    {'profession_id': 4, 'profession_name': 'Project Manager'},
    {'profession_id': 5, 'profession_name': 'Marketing Specialist'}
]

# Генерируем для профессий 
lesson_data = []
lesson_id = 1
for profession in professions:
    for i in range(1, 4):
        lesson_data.append({
            'lesson_id': lesson_id,
            'lesson_name': fake.word(),
            'profession_id': profession['profession_id'],
            'profession_name': profession['profession_name']
        })
        lesson_id += 1

# Делаем для 200 юзеров
finished_lesson_data = []
user_ids = list(range(1, 201)) 
completed_lessons = set() 
lesson_counter = 1
for user_id in user_ids:
    for lesson_id in range(1, lesson_id):
        profession_id = lesson_data[lesson_id - 1]['profession_id'] 
        if (lesson_id, user_id) not in completed_lessons: 
            lesson_date = fake.date_time_between(start_date='-30d', end_date='now')
            finished_lesson_data.append({
                'id': lesson_counter,
                'date_created': lesson_date,
                'lesson_id': lesson_id,
                'user_id': user_id
            })
            lesson_counter += 1
            completed_lessons.add((lesson_id, user_id)) 
            if random.random() < 0.2 and lesson_id < lesson_counter:  # 20% probability
                prev_lesson_date = lesson_date - timedelta(seconds=random.uniform(0, 5))
                finished_lesson_data.append({
                    'id': lesson_counter,
                    'date_created': prev_lesson_date,
                    'lesson_id': lesson_id,
                    'user_id': user_id
                })
                lesson_counter += 1

lesson_df = pd.DataFrame(lesson_data)
finished_lesson_df = pd.DataFrame(finished_lesson_data)


In [7]:
merged_df = pd.merge(
    finished_lesson_df, lesson_df[['lesson_id', 'lesson_name', 'profession_id']], on='lesson_id', how='left')
merged_df.sort_values(by=['user_id', 'profession_id', 'date_created'], inplace=True)

# Calculate the time delta between consecutive lessons for each user
merged_df['completion_time'] = merged_df.groupby(['user_id', 'profession_id'])['date_created'].diff().dt.seconds
merged_df.to_csv('merged_df.csv', index=False)

## Ссылка на борд в Tableau 
https://public.tableau.com/app/profile/ekaterina.kolodyazhnaya3944/viz/testtask2_17141612964500/TripleTen